<a href="https://colab.research.google.com/github/rogeliorv/encode-ai-bootcamp-g11/blob/main/Gethsun_WeekendI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive AI Chef: Recipe Suggestions, Critiques, and Ingredient-Based Dish Ideas

*This Google Colab notebook provides an interactive AI chef experience. Users can input ingredients to get dish suggestions, request detailed recipes for specific dishes, or submit their own recipes for critique and improvement. The AI, modeled as a young and spirited Kenyan cook, offers clear, detailed, and helpful responses to enhance your cooking experience.*

In [ ]:
!pip install openai

In [15]:
from openai import OpenAI
from google.colab import userdata # Import the userdata module from Google Colab
userdata.get('OPEN_API_KEY') # Retrieve your API key (replace 'OPEN_API_KEY' with the name you used to store it)

api_key = "OPEN_API_KEY"
client = OpenAI(api_key=api_key) # Initialize the OpenAI client with the API key

In [ ]:
# Initial system messages to set the context and behavior of the AI
messages = [
    {
        "role": "system",
        "content": (
            "You are a young and spirited Kenyan cook that loves to make Githeri. "
            "You help people by suggesting detailed recipes for dishes they want to cook, "
            "suggesting dishes based on ingredients provided, and criticizing recipes given by the user. "
            "You always try to be as clear as possible and provide the best possible suggestions for the user's needs. "
            "You know a lot about different cuisines and cooking techniques. You are also very patient and understanding with the user's needs and questions."
        ),
    },
    {
        "role": "system",
        "content": (
            "Your client will provide either ingredients, a dish name, or a recipe. "
            "If the input is a list of ingredients, suggest a dish name that can be made with those ingredients, but do not provide the recipe at this stage. "
            "If the input is a dish name, provide a detailed recipe for that dish. "
            "If the input is a recipe, critique the recipe and suggest improvements. "
            "If the input does not match these scenarios, deny the request and ask the user to try again."
        ),
    }
]

# Get the user's input
user_input = input("Enter your request (ingredients, dish name, or recipe):\n")

# Determine the type of input: ingredients, dish, or recipe
if ',' in user_input:
    input_type = "ingredients"
elif any(char.isdigit() for char in user_input):
    input_type = "recipe"
else:
    input_type = "dish"

# Respond based on input type
if input_type == "ingredients":
    messages.append(
        {
            "role": "user",
            "content": f"Suggest a dish name that can be made with these ingredients: {user_input}"
        }
    )
elif input_type == "dish":
    messages.append(
        {
            "role": "user",
            "content": f"Suggest me a detailed recipe and the preparation steps for making {user_input}"
        }
    )
elif input_type == "recipe":
    messages.append(
        {
            "role": "user",
            "content": f"Criticize this recipe and suggest changes: {user_input}"
        }
    )
else:
    print("Invalid input. Please enter a list of ingredients, a dish name, or a recipe.")
    exit()

# Define the model to use for the completion
model = "gpt-3.5-turbo"

# Create a stream to get the completion from the model
stream = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=True,
)

# Print the response from the model in real-time
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

# Collect the messages from the stream
collected_messages = []
for chunk in stream:
    chunk_message = chunk.choices[0].delta.content or ""
    print(chunk_message, end="")
    collected_messages.append(chunk_message)

# Append the response to the messages list
messages.append(
    {
        "role": "system",
        "content": "".join(collected_messages)
    }
)

# Loop to keep taking input and generating responses
while True:
    print("\n")
    user_input = input()  # Get user input
    # Determine the type of input: ingredients, dish, or recipe
    if ',' in user_input:
        input_type = "ingredients"
    elif any(char.isdigit() for char in user_input):
        input_type = "recipe"
    else:
        input_type = "dish"

    # Respond based on input type
    if input_type == "ingredients":
        messages.append(
            {
                "role": "user",
                "content": f"Suggest a dish name that can be made with these ingredients: {user_input}"
            }
        )
    elif input_type == "dish":
        messages.append(
            {
                "role": "user",
                "content": f"Suggest me a detailed recipe and the preparation steps for making {user_input}"
            }
        )
    elif input_type == "recipe":
        messages.append(
            {
                "role": "user",
                "content": f"Criticize this recipe and suggest changes: {user_input}"
            }
        )
    else:
        print("Invalid input. Please enter a list of ingredients, a dish name, or a recipe.")
        continue

    # Create a stream to get the completion from the model
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )

    # Collect the messages from the stream and print the response in real-time
    collected_messages = []
    for chunk in stream:
        chunk_message = chunk.choices[0].delta.content or ""
        print(chunk_message, end="")
        collected_messages.append(chunk_message)

    # Append the response to the messages list
    messages.append(
        {
            "role": "system",
            "content": "".join(collected_messages)
        }
    )
